In [44]:
# first level direction: alwasy left
# generate clock wise
def xy2d(n, x, y):
    d = 0
    s = int(n / 2)
    while s > 0:
        rx = (x & s) > 0
        ry = (y & s) > 0
        if rx == 0 and ry == 0:
            d += 0
        if rx == 1 and ry == 0:
            d += s*s
        if rx == 1 and ry == 1:
            d += s*s*2
        if rx == 0 and ry == 1:
            d += s*s*3   
#         d += s * s * ((3 * rx) ^ ry)
        x,y = rot(s, x, y, rx, ry)
        s = int(s/2)
    return d

# def d2xy(n, d):
#     t = d
#     x = 0
#     y = 0
#     s = 1
#     while s < n:
#         rx = 1 & int(t/2)
#         ry = 1 & (t^rx)
#         x,y = rot(s, x, y, rx, ry)
#         x += s * rx
#         y += s * ry
#         t = int(t/4)
#         s *= 2
#     return x, y

def rot(n, x, y, rx, ry):
    if rx == 0 : # the upper level direction remains the same, so only change the lower level
        if ry == 1 :
            x = n-1 - x
            y = n-1 - y
        t  = x
        x = y
        y = t
    return x, y

# ===== Testing =====
# for i in range(8):
#     for j in range(8):
#         print(xy2d(8, i, j))

In [45]:
# import ipdb
# ipdb.set_trace()
# print(xy2d(1<<14, 2506754>>8, 3194880>>8))
# print(xy2d(1<<14, 2818050>>8, 1785859>>8))
# print(xy2d(1<<14, 2506756>>8, 3194880>>8))

In [46]:
from enum import Enum
class Direction(Enum):
    UP = 0
    LEFT = 1
    RIGHT = 2
    DOWN = 3

class ClockDirection(Enum):
    CLOCKWISE = 0
    COUNTERCLOCKWISE = 1
    UNAVAILABLE = 2

class Quadrant(Enum):
    LOWERLEFT = 0
    LOWERRIGHT = 1
    UPPERRIGHT = 2
    UPPERLEFT = 3
    
next_level_direction=[[Direction.UP, Direction.UP, Direction.RIGHT, Direction.LEFT],
                      [Direction.DOWN, Direction.LEFT, Direction.LEFT, Direction.UP],
                      [Direction.RIGHT, Direction.DOWN, Direction.UP, Direction.RIGHT],
                      [Direction.LEFT, Direction.RIGHT, Direction.DOWN, Direction.DOWN]]

next_level_clock_direction = [[ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE],
                           [ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.CLOCKWISE],
                           [ClockDirection.CLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE],
                           [ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE]]

# ===== Testing =====
# for i in range(4):
#     for j in range(4):
#         print(next_level_direction[i][j])
# vard = Direction.UP
# if vard.name == "UP":
#     print("yes") # yes
# print(vard.name)
# print(vard.value)
# if vard == Direction.UP:
#     print("yes")
# for dir in Direction:
#     print(dir)

In [47]:
# dictionary for interval:
# lower_x
# upper_X
# lower_y
# upper_y
# lower_hilbert
# upper_hilbert


def range_decompose(max_bits, current_bits, down_x, up_x, down_y, up_y, center_x, center_y, pre_dir, this_dir, intervals):
    
    # if there is only 1 element here
    if current_bits == 0:
        hvalue = xy2d(1<<max_bits, down_x, down_y)
        interval = {"lower_x":down_x, "upper_x":up_x, "lower_y":down_y,"upper_y":up_y,"lower_hilbert":hvalue,"upper_hilbert":hvalue}
        intervals.append(interval);
        return
    
    half_region = 1 << (current_bits - 1)
    clock_dir = next_level_clock_direction[pre_dir.value][this_dir.value]
    if current_bits == max_bits:
        clocl_dir = ClockDirection.COUNTERCLOCKWISE
        
    # check if the range contains the entire region
    if down_x == center_x - half_region and down_y == center_y - half_region and up_x == center_x + half_region-1 and up_y == center_y + half_region-1:
        # if yes, return this region as an interval
        lower_x = 0
        upper_x = 0
        lower_y = 0
        upper_y = 0
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                lower_x = up_x
                upper_x = down_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "DOWN":
                lower_x = down_x
                upper_x = up_x
                lower_y = down_y
                upper_y = down_y
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                lower_x = down_x
                upper_x = up_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "DOWN":
                lower_x = up_x
                upper_x = down_x
                lower_y = down_y
                upper_y = down_y
                
        lowhvalue = xy2d(1<<max_bits,lower_x,lower_y)
        uphvalue = xy2d(1<<max_bits,upper_x,upper_y)
        interval = {"lower_x":lower_x, "upper_x":upper_x, "lower_y":lower_y,"upper_y":upper_y,"lower_hilbert":lowhvalue,"upper_hilbert":uphvalue}
        intervals.append(interval)
    else:
        # if not, further decompose it
        current_bits -= 1
        if current_bits <= 0:
            half_region = 0
        else:
            half_region = 1 << (current_bits - 1)
        
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)

# ===== Testing =====
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,0,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.DOWN,Direction.DOWN,intervals)
# print(intervals)

# import ipdb
# ipdb.set_trace()
# intervals = []
# range_decompose(3,3,0,3,0,7,4,4,Direction.LEFT,Direction.LEFT,intervals)
# print(len(intervals))
# intervals = intervals[0]
# print(intervals[0])
# print(intervals[0]["lower_x"])
# for inter in intervals:
#     print(intervals)

In [48]:
def merge_intervals(intervals):
    if len(intervals) <= 1:
        return intervals
    merged_intervals = []
    i = 0
    j = 0
    while i < len(intervals):
        temp_upper = intervals[i]["upper_hilbert"]
        j = 1
        while i+j < len(intervals) and intervals[i+j]["lower_hilbert"] == temp_upper + 1:
            temp_upper = intervals[i+j]["upper_hilbert"]
            j += 1
        interval = {"lower_x":intervals[i]["lower_x"], "upper_x":intervals[i+j-1]["upper_x"], 
                    "lower_y":intervals[i]["lower_y"],"upper_y":intervals[i+j-1]["upper_y"],
                    "lower_hilbert":intervals[i]["lower_hilbert"],"upper_hilbert":intervals[i+j-1]["upper_hilbert"]}
        merged_intervals.append(interval)
        i += j
    return merged_intervals

def get_intervals(lower_x, upper_x, lower_y, upper_y, max_bits = 22):
    intervals = []
    range_decompose(max_bits,max_bits,lower_x,upper_x,lower_y,upper_y,1<<(max_bits-1),1<<(max_bits-1),Direction.LEFT,Direction.LEFT,intervals)
    intervals = merge_intervals(intervals)
    return intervals
    
# ===== Testing =====
#print(get_intervals(2,4,2,4))

In [49]:
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# print(intervals)
# intervals = merge_intervals(intervals)
# print(intervals)

In [50]:
# print(xy2d(4194306, 1600000, 1900000))

In [51]:
import pandas as pd
df = pd.read_csv("./Data/HilbertSortedPOIs.csv")
print(df)
TOTAL = len(df)+1
COL = df.columns

import numpy as np
print(np.any(np.isnan(df))) # should be false
print(np.all(np.isfinite(df))) # should be true
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
print(scaler.fit(df.iloc[:,0:5]))
print(scaler.mean_)
df.iloc[:,0:5]=scaler.transform(df.iloc[:,0:5])
print(df)

# ===== Testing =====
# df1 = pd.DataFrame(data=[[-76.3099,22.4995,273802,2024995,17344470968016]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df1 = pd.DataFrame(data=[[-10,10,1600000,1900000,2819270465194]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# print(df1)
# df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
# print(df1)

               x         y    x_     y_    hilbert    order  prediction
0       -83.8626 -160.3120   479    769     850940        0         NaN
1       -47.1500 -133.7760  3347   1805    7121756        1         NaN
2       -57.8599  -80.3684  2510   3891   12476251        2         NaN
3       -85.0511  -95.9735   386   3282   15937800        3         NaN
4       -85.0511  -96.0033   386   3281   15937805        4         NaN
5       -85.0494  -80.8804   386   3871   16564569        5         NaN
6       -66.5151  -39.3818  1834   5492   19274204        6         NaN
7       -66.5134  -39.3750  1834   5493   19274205        7         NaN
8       -82.2142   11.2500   608   7470   22919422        8         NaN
9       -76.3099   22.4995  1069   7910   23394509        9         NaN
10      -54.9371  -67.6341  2739   4389   32367030       10         NaN
11      -54.9356  -67.6150  2739   4390   32367033       11         NaN
12      -54.7994  -68.2801  2750   4364   32370444       12     

In [52]:
from sklearn import linear_model
from sklearn import neural_network
from sklearn import ensemble
import ipdb
import time
stages = [1,10,100]
index = []
data = []

start_time=time.time()

for i in range(len(stages)):
    temp = []
    for j in range(stages[i]):
        df_ = pd.DataFrame(columns=COL)
        temp.append(df_)
    data.append(temp)
    
data[0][0] = df
nodatacount = 0

for i in range(len(stages)):
    temp = []
    print("======stage:"+str(i)+"======")
    
    for j in range(stages[i]):
        print("model:"+str(j))
        if len(data[i][j]) == 0:
            nodatacount += 1
            data[i][j] = data[i][j-1]
            print(j)
#         lr = linear_model.LinearRegression()
#         lr = linear_model.ElasticNet(max_iter=10000)
#         lr = linear_model.Lasso(max_iter=10000)
        lr = ensemble.GradientBoostingRegressor()
#         lr = ensemble.AdaBoostRegressor()
#         if i == 0:
# #             lr = neural_network.MLPRegressor(hidden_layer_sizes=(8, 2),max_iter=1000)
#             lr = ensemble.GradientBoostingRegressor(n_estimators=500)
#         else:
#             lr = ensemble.GradientBoostingRegressor()
        train_x = data[i][j].iloc[:,4].reshape(-1,1)
#         train_x = data[i][j].iloc[:,0:5]
        train_y = data[i][j].iloc[:,5]
        lr.fit(train_x,train_y)
        temp.append(lr)
        # allocate data
        if i < len(stages)-1:
            predicted = lr.predict(train_x)
            for pi in range(len(predicted)): # do not change the prediction
                #print(item)
                if predicted[pi] < 0:
                    predicted[pi] = 0
                if predicted[pi] >= TOTAL:
                    predicted[pi] = TOTAL-1   
            ModelID = ((predicted/TOTAL)*stages[i+1]).astype(int)
            data[i][j].iloc[:,-1]=ModelID
#             if i == 1 and np.any(data[i][j]['prediction'] == 4):
#                 print(j)

            for k in range(stages[i+1]):
                data[i+1][k] = data[i+1][k].append(data[i][j].loc[data[i][j]['prediction'] == k])
    index.append(temp)

elapsed_time = time.time() - start_time
print(elapsed_time)
print(nodatacount)

======stage:0======
model:0


D:\anaconda\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


======stage:1======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
======stage:2======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
model:10
model:11
model:12
model:13
model:14
model:15
model:16
model:17
model:18
model:19
model:20
model:21
model:22
model:23
model:24
model:25
model:26
model:27
model:28
model:29
model:30
model:31
model:32
model:33
model:34
model:35
model:36
model:37
model:38
model:39
model:40
model:41
model:42
model:43
model:44
model:45
model:46
model:47
model:48
model:49
model:50
model:51
model:52
model:53
model:54
model:55
model:56
model:57
model:58
model:59
model:60
model:61
model:62
model:63
model:64
model:65
model:66
model:67
model:68
model:69
model:70
model:71
model:72
model:73
model:74
model:75
model:76
model:77
model:78
model:79
model:80
model:81
model:82
model:83
model:84
model:85
model:86
model:87
model:88
model:89
model:90
model:91
model:92
model:93
model:94
model:95
model:96
model:97
model:98

In [53]:
import numpy as np
# use the generated stage index for prediction
def predict_single_item_with_stage_model(model, data, total_size):
    tempy = 0
    modelIndex = 0
    for m in range(len(model)):
        tempy = model[m][modelIndex].predict(data)
#         tempy = model[m][modelIndex].predict(data.iloc[i,:].reshape(1,-1))
#         tempy = model[m][modelIndex].predict(data.iloc[i,])
        if tempy < 0:
            tempy = 0
        if tempy >= total_size:
            tempy = total_size-1
        if m < len(model)-1:
            modelIndex = int((tempy/total_size)*len(model[m+1]))
    return tempy

In [54]:
def find_discrepancy(hilbert_value, df, position):
    true_hilbert_value = df.iloc[position,4]
    dif_first = 0
    dif_next = 0
    dif_first = abs(true_hilbert_value-hilbert_value)
    if dif_first == 0:
        return 0
    cost = 1
    while True:
        if true_hilbert_value < hilbert_value:
            true_hilbert_value = df.iloc[position+cost,4]
            dif_next = abs(true_hilbert_value-hilbert_value)
            if dif_next > dif_first:
                return cost-1
            else:
                dif_first = dif_next
                cost += 1
        else:
            true_hilbert_value = df.iloc[position-cost,4]
            dif_next = abs(true_hilbert_value-hilbert_value)
            if dif_next > dif_first:
                return cost-1
            else:
                dif_first = dif_next
                cost += 1

In [55]:
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
# cost1 = find_discrepancy(-2.103575, df, 0)
# print(cost1)

In [56]:
def range_query(lower_x, upper_x, lower_y, upper_y,max_bits = 22, print_intervals = False, print_cost = False, model = None, df = None, scaler = None):
    lx = int((2*lower_x+180)*10000) >> 22-max_bits
    ly = int((lower_y+180)*10000) >> 22-max_bits
    ux = int((2*upper_x+180)*10000) >> 22-max_bits
    uy = int((upper_y+180)*10000) >> 22-max_bits
    
    intervals = get_intervals(lx,ux,ly,uy,max_bits)
    total_size = len(intervals)
    print("total intervals: ",total_size)
    if print_intervals:
        print(intervals)
    if print_cost:
        cost = 0
        for interval in intervals:
            
            df1 = pd.DataFrame(data=[[lower_x,lower_y,lx,ly,interval["lower_hilbert"]]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
            df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
            predicted_position1 = int(predict_single_item_with_stage_model(index, df1.iloc[0,4], len(df))[0])
            
            df2 = pd.DataFrame(data=[[upper_x,upper_y,ux,uy,interval["upper_hilbert"]]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
            df2.iloc[0,:] = scaler.transform(df2.iloc[0,:].reshape(1,-1))
            predicted_position2 = int(predict_single_item_with_stage_model(index, df2.iloc[0,4], len(df))[0])
            
            cost1 = find_discrepancy(df1.iloc[0,4], df, predicted_position1)
            cost2 = find_discrepancy(df2.iloc[0,4], df, predicted_position2)
            
            cost += cost1
            cost += cost2
        print("total cost: ",cost)     

In [57]:
# import ipdb
# ipdb.set_trace()
# range_query(1, 2, 1, 2,14)
# range_query(1, 10, 1, 10,14)
# range_query(-10, 10, -10, 10,14)
# range_query(-20, 20, -20, 20,14)
# range_query(-30, 30, -30, 30,14)
# range_query(-40, 40, -40, 40,14)
# range_query(-15.1234,17.1983,-13.1677,11.8765,14)

# range_query(38.7900,39.0118,-77.2123,-76.8835,14) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,14) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,14) # Hong Kong island
# import ipdb
# ipdb.set_trace()
# range_query(38.0000,38.0010,-77.2123,-77.2113,22,False,True,index,df,scaler) # For Test

# range_query(38.7900,39.0118,-77.2123,-76.8835,22,False,True,index,df,scaler) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,22,False,True,index,df,scaler) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,22,False,True,index,df,scaler) # Hong Kong island

range_query(38.7900,39.0118,-77.2123,-76.8835,14,False,True,index,df,scaler) # Washington DC
range_query(22.1481,22.5457,113.8436,114.4379,14,False,True,index,df,scaler) # Hong Kong
range_query(22.2201,22.2841,114.1278,114.2572,14,False,True,index,df,scaler) # Hong Kong island

total intervals:  16
total cost:  14413
total intervals:  31
total cost:  620
total intervals:  8
total cost:  3914


D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\anaconda\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [41]:
import csv
# HILBERT_N = 4194306 # 22 bits, 4 Million
HILBERT_N = 16384 # 14 bits
list = []
count = -1
with open("./Data/SortedPOIs.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if count == -1: # ignore the head info
            count += 1
            continue
        x = float(row[0])
        y = float(row[1])
        x_ = int((2*x+180)*10000)>>8
        y_ = int((y+180)*10000)>>8
        h_ = xy2d(HILBERT_N,x_, y_)
        list.append([x,y,x_,y_,h_,count])
        count += 1
        if count % 10000 == 0:
            print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000


In [42]:
import csv
def write_csv_file(path, head, data):  
    try:  
        with open(path, 'w', newline='') as csv_file:  
            writer = csv.writer(csv_file, dialect='excel')  
  
            if head is not None:  
                writer.writerow(head)  
  
            for row in data:  
                writer.writerow(row)  
  
            print("Write a CSV file to path %s Successful." % path)  
    except Exception as e:  
        print("Write an CSV file to path: %s, Case: %s" % (path, e)) 

In [43]:
list.sort(key = lambda x:(x[4]))
for i in  range(len(list)):
    list[i][5] = i
write_csv_file("./Data/HilbertSortedPOIs.csv",['x','y','x_','y_','hilbert','order','prediction'],list)

Write a CSV file to path ./Data/HilbertSortedPOIs.csv Successful.
